In [1]:
"""
    Code adapted from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
"""


%matplotlib tk


Transfer Learning tutorial
==========================
**Author**: `Sasank Chilamkurthy <https://chsasank.github.io>`_

In this tutorial, you will learn how to train your network using
transfer learning. You can read more about the transfer learning at `cs231n
notes <http://cs231n.github.io/transfer-learning/>`__

Quoting these notes,

    In practice, very few people train an entire Convolutional Network
    from scratch (with random initialization), because it is relatively
    rare to have a dataset of sufficient size. Instead, it is common to
    pretrain a ConvNet on a very large dataset (e.g. ImageNet, which
    contains 1.2 million images with 1000 categories), and then use the
    ConvNet either as an initialization or a fixed feature extractor for
    the task of interest.

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initializaion, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.




In [2]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import time
import os
import copy


plt.ion()   # interactive mode

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [3]:
# -- TODO
from models import get_model

experiments_path = "experiments/"
data_dir = 'datasets/'


def save_model(model, acc, data_transforms, folder, file_name, other_info=dict()):
    save_data = {
        "state_dict": model.state_dict(),
        "data_transforms": data_transforms,
        "acc": acc,
        "model_name": model_name,
    }
    save_data.update(other_info)
    
    torch.save(save_data, os.path.join(folder, f"checkpoint_{file_name}"))


def new_save_folder(exp_name):
    save_f = os.path.join(experiments_path, f"{int(time.time())}_{exp_name}")
    os.mkdir(save_f)
    return save_f

    
model_name = "alexnet"

in_size = torch.Size([3, 224, 224])
out_size = None  # Calculate after loading data
best_eval_acc = 0

# Create experiment folder
save_best = True
save_all = True

In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
out_size = torch.Size([len(class_names)])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



Visualize a few images
^^^^^^^^^^^^^^^^^^^^^^
Let's visualize a few training images so as to understand the data
augmentations.



In [5]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [6]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
from util import report_net_param

plt.ion()


def plot_multi(fig_data, info):
    if fig_data is None:
        fig = plt.figure()
        no_plots = len(info)
        ax = fig.subplots((no_plots - 1) // 2 + 1, 2)
    else:
        fig, ax = fig_data

    for ix, (plot_name, traces) in enumerate(info.items()):
        c_ax = ax[ix//2, ix%2]
        c_ax.clear()
        c_ax.title.set_text(plot_name)
        for trace_name, (x, y) in traces.items():
            c_ax.plot(x, y, label=trace_name)
        c_ax.legend(loc=3)

    fig.canvas.draw()
    return (fig, ax)

def append_to_info(data, step, value):
    data[0].append(step)
    data[1].append(value)

fig_data = None
do_plot = True



Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [7]:
def train_model(model, criterion, optimizer, scheduler, save_folder, other_info, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    all_batch_cnt = 0
    
    if do_plot:
        fig_data = None
        info = {
            "acc": {"train": (list(), list()), "val": (list(), list())},
            "loss": {"train": (list(), list()), "val": (list(), list())},
            "param": {"max": (list(), list()), "avg": (list(), list())},
            "grad": {"max": (list(), list()), "avg": (list(), list())},
            "output": {"min": (list(), list()), "max": (list(), list()), "entropy": (list(), list())},
        }

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                        # statistics to plot
                        if do_plot:
                            pred_max = outputs.max().item()
                            pred_min = outputs.min().item()
                            norm = torch.nn.Softmax(1)(outputs.detach())
                            pred_entropy = torch.sum(- norm * norm.log(), dim=1).mean().item()
                            append_to_info(info["output"]["max"], all_batch_cnt, pred_max)
                            append_to_info(info["output"]["min"], all_batch_cnt, pred_min)
                            append_to_info(info["output"]["entropy"], all_batch_cnt, pred_entropy)

                            # Network info
                            param_max, param_avg, grad_max, grad_avg = report_net_param(model)
                            append_to_info(info["param"]["max"], all_batch_cnt, param_max)
                            append_to_info(info["param"]["avg"], all_batch_cnt, param_avg)
                            append_to_info(info["grad"]["max"], all_batch_cnt, grad_max)
                            append_to_info(info["grad"]["avg"], all_batch_cnt, grad_avg)
                        
                        all_batch_cnt += 1

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
                if save_best:
                    save_model(model, epoch_acc, data_transforms, save_folder, "best", other_info)

            if save_all:
                save_model(model, epoch_acc, data_transforms, save_folder, str(epoch), other_info)

            # statistics to plot
            if do_plot:
                append_to_info(info["acc"][phase], epoch, epoch_acc)
                append_to_info(info["loss"][phase], epoch, epoch_loss)


        if do_plot:
            fig_data = plot_multi(fig_data, info)
            
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

Visualizing the model predictions
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Generic function to display predictions for a few images




In [8]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

Finetuning the convnet
----------------------

Load a pretrained model and reset final fully connected layer.




In [9]:
model_ft = get_model(model_name, in_size=in_size, out_size=out_size, pretrained=True)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Not loading layer: classifier.6.weight
Not loading layer: classifier.6.bias


Train and evaluate
^^^^^^^^^^^^^^^^^^

It should take around 15-25 min on CPU. On GPU though, it takes less than a
minute.




In [10]:
save_folder = new_save_folder("summer_full_train")
other_info = {"in_size": in_size, "out_size": out_size, "model_name": model_name}
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, save_folder, other_info,
                       num_epochs=5)

Epoch 0/4
----------
train Loss: 0.0750 Acc: 0.9720
val Loss: 0.0000 Acc: 1.0000

Epoch 1/4
----------
train Loss: 0.0000 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000

Epoch 2/4
----------


Process Process-24:
Process Process-22:
Process Process-23:
Process Process-21:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/andrei/anaconda2/envs/old_pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/andrei/anaconda2/envs/old_pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/andrei/anaconda2/envs/old_pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/andrei/anaconda2/envs/old_pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/andrei/anaconda2/envs/old_pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/andrei/anaconda2/envs/old_pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run


KeyboardInterrupt: 

In [ ]:
visualize_model(model_ft)

ConvNet as fixed feature extractor
----------------------------------

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <http://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [10]:
model_conv = get_model(model_name, in_size=in_size, out_size=out_size, pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
# Custom for alexnet 
num_ftrs3 = model_conv.classifier[-3].in_features
num_ftrs1 = model_conv.classifier[-1].in_features
model_conv.classifier[-3] = nn.Linear(num_ftrs3, num_ftrs1)
model_conv.classifier[-1] = nn.Linear(num_ftrs1, out_size[0])

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.classifier[-1].parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

Not loading layer: classifier.6.weight
Not loading layer: classifier.6.bias


Train and evaluate
^^^^^^^^^^^^^^^^^^

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.




In [11]:
save_folder = new_save_folder("summer_fine_tune")
other_info = {"in_size": in_size, "out_size": out_size, "model_name": model_name}
model_conv = train_model(model_conv, criterion, optimizer_conv, 
                         exp_lr_scheduler, save_folder, other_info, num_epochs=25)

Epoch 0/24
----------
tensor(1.00000e-02 *
       1.4880, device='cuda:0')
tensor(1.4196, device='cuda:0')
tensor(1.00000e-02 *
       1.4816, device='cuda:0')
tensor(1.4196, device='cuda:0')
tensor(1.00000e-02 *
       1.4769, device='cuda:0')
tensor(1.4196, device='cuda:0')
tensor(1.00000e-02 *
       1.5038, device='cuda:0')
tensor(1.4196, device='cuda:0')
tensor(1.00000e-02 *
       1.5422, device='cuda:0')
tensor(1.4196, device='cuda:0')
tensor(1.00000e-02 *
       1.5571, device='cuda:0')
tensor(1.4197, device='cuda:0')
tensor(1.00000e-02 *
       1.5523, device='cuda:0')
tensor(1.4197, device='cuda:0')
tensor(1.00000e-02 *
       1.5956, device='cuda:0')
tensor(1.4197, device='cuda:0')
tensor(1.00000e-02 *
       1.6405, device='cuda:0')
tensor(1.4198, device='cuda:0')
tensor(1.00000e-02 *
       1.6662, device='cuda:0')
tensor(1.4198, device='cuda:0')
tensor(1.00000e-02 *
       1.6382, device='cuda:0')
tensor(1.4199, device='cuda:0')
tensor(1.00000e-02 *
       1.5969, device=

tensor(1.00000e-02 *
       1.9272, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9271, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9258, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9249, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9229, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9203, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9164, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9134, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9108, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9098, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9093, device='cuda:0')
tensor(1.4219, device='cuda:0')
tensor(1.00000e-02 *
       1.9084, device='cuda:0')
tensor(1.421

tensor(1.00000e-02 *
       2.0082, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       2.0059, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       2.0026, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       1.9991, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       1.9953, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       1.9915, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       1.9880, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       1.9771, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       1.9673, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       1.9568, device='cuda:0')
tensor(1.4222, device='cuda:0')
tensor(1.00000e-02 *
       1.9471, device='cuda:0')
tensor(1.4223, device='cuda:0')
tensor(1.00000e-02 *
       1.9385, device='cuda:0')
tensor(1.422

KeyboardInterrupt: 

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()